In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import (
    VERSION_LAST,
    FuncGroups,
    InlineType,
    Version,
    SymbolTable,
)

from utils import OUTPUT_PATH


def count_inline(v: Version):
    groups = FuncGroups.from_dump(v.dwarf_funcs_path)
    results = {t: FuncGroups() for t in InlineType}
    symtab = SymbolTable.from_dump(v.symtab_path)

    for name, group in groups.iter_groups():
        in_symtab = name in symtab.func_sym_groups
        inline_type = group.get_inline_type(in_symtab=in_symtab)
        results[inline_type].add_group(name, group)

    for t, group in results.items():
        group.save_funcs(OUTPUT_PATH / "inline" / v.name / f"{t.name.lower()}.txt")

    total = groups.num_groups

    return {t: group.num_groups / total for t, group in results.items()}


# count_inline(VERSION_LAST)

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.


In [3]:
from depsurf import VersionGroup
from utils import save_pkl

groups = [VersionGroup.REGULAR, VersionGroup.ARCH]

data = {}
for group in groups:
    group_data = {}
    for version in group.versions:
        group_data[version] = count_inline(version)
    data[group] = group_data

save_pkl(data, "inline")

[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.4.0-21-generic-amd64.jsonl


[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.4.0-21-generic-amd64.jsonl
[        group.py:75 ] WARNING: xen_pvh_early_cpu_init has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: __builtin_strlen has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: relocate_kernel has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: __builtin_strchr has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: swsusp_arch_suspend has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.py:75 ] WARNING: __builtin_strcpy has func caller but no sym entry. It is likely a builtin function or a ELF-related function.
[        group.p

In [4]:
import pandas as pd

from utils import load_pkl

data = load_pkl("inline")
pd.DataFrame(
    {
        (group, version): data[group][version]
        for group in data
        for version in data[group]
    }
).T

[          pkl.py:18 ] INFO: Loding inline from /Users/szhong/Downloads/bpf-study/output/inline.pkl


Not inlined  Fully inlined  Partially inlined
Regular 4.4.0-21-generic-amd64       0.543473       0.346672           0.109855
        4.8.0-22-generic-amd64       0.543648       0.346881           0.109472
        4.10.0-19-generic-amd64      0.542314       0.348975           0.108711
        4.13.0-16-generic-amd64      0.534523       0.350582           0.114896
        4.15.0-20-generic-amd64      0.536402       0.348616           0.114983
        4.18.0-10-generic-amd64      0.547821       0.350536           0.101643
        5.0.0-13-generic-amd64       0.545583       0.354027           0.100390
        5.3.0-18-generic-amd64       0.548839       0.353256           0.097905
        5.4.0-26-generic-amd64       0.547360       0.354456           0.098184
        5.8.0-25-generic-amd64       0.568490       0.327451           0.104059
        5.11.0-16-generic-amd64      0.576946       0.322732           0.100322
        5.13.0-19-generic-amd64      0.555827       0.347098           0.097075
        5.15.0-25-generic-amd64      0.556735       0.348744           0.094521
        5.19.0-21-generic-amd64      0.561719       0.349475           0.088806
        6.2.0-20-generic-amd64       0.558719       0.353250           0.088030
        6.5.0-9-generic-amd64        0.560868       0.350575           0.088557
        6.8.0-31-generic-amd64       0.560232       0.350305           0.089463
Arch    5.4.0-26-generic-arm64       0.552155       0.350248           0.097597
        5.4.0-26-generic-armhf       0.567610       0.344633           0.087757
        5.4.0-26-generic-ppc64el     0.554854       0.355316           0.089830
        5.4.0-26-generic-s390x       0.512261       0.375598           0.112141

In [7]:
import numpy as np
from matplotlib import pyplot as plt

from depsurf import InlineType
from utils import get_legend_handles_labels, load_pkl, save_fig

data = load_pkl("inline")

COLORS = {
    InlineType.NOT: "tab:blue",
    InlineType.FULL: "tab:green",
    InlineType.PARTIAL: "tab:red",
}

GROUPS = {
    VersionGroup.REGULAR: "Linux Kernel Version",
    VersionGroup.ARCH: "Arch for v5.4",
}

fig, axs = plt.subplots(
    1,
    len(data),
    figsize=(5, 1.25),
    sharey=True,
    width_ratios=[len(v) for v in data.values()],
    gridspec_kw={"wspace": 0.1},
)

for group_idx, (ax, (group, group_data)) in enumerate(zip(axs, data.items())):
    for i, (version, version_data) in enumerate(group_data.items()):
        bottom = 0
        for k, v in version_data.items():
            ax.bar(i, v, 0.7, label=k, bottom=bottom, color=COLORS[k])
            bottom += v

    labels = group.labels
    ax.set_xticks(np.arange(len(labels)), labels)
    ax.set_xlabel(GROUPS[group], fontsize=8)
    ax.set_xticklabels(labels, fontsize=8, rotation=90)

    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0%}"))

axs[0].set_ylabel("Pct. of Functions")

handles, labels = get_legend_handles_labels(fig)
fig.legend(
    handles,
    labels,
    loc="upper center",
    ncol=5,
    bbox_to_anchor=(0.5, 1.1),
    fontsize=8,
)

save_fig(fig, "inline")

[          pkl.py:18 ] INFO: Loding inline from /Users/szhong/Downloads/bpf-study/output/inline.pkl
[          mpl.py:84 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/inline.pdf
